In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from secret import *
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(
    url = "bolt://localhost:7687",
    username = "neo4j",
    password = "snomed1234"
)

In [11]:
graph.structured_schema

{'node_props': {'ObjectConcept': [{'property': 'nodetype', 'type': 'STRING'},
   {'property': 'history', 'type': 'STRING'},
   {'property': 'moduleId', 'type': 'STRING'},
   {'property': 'definitionStatusId', 'type': 'STRING'},
   {'property': 'effectiveTime', 'type': 'STRING'},
   {'property': 'id', 'type': 'STRING'},
   {'property': 'sctid', 'type': 'STRING'},
   {'property': 'FSN', 'type': 'STRING'},
   {'property': 'active', 'type': 'STRING'}],
  'Description': [{'property': 'typeId', 'type': 'STRING'},
   {'property': 'term', 'type': 'STRING'},
   {'property': 'id128bit', 'type': 'STRING'},
   {'property': 'descriptionType', 'type': 'STRING'},
   {'property': 'refsetId', 'type': 'STRING'},
   {'property': 'acceptabilityId', 'type': 'STRING'},
   {'property': 'caseSignificanceId', 'type': 'STRING'},
   {'property': 'moduleId', 'type': 'STRING'},
   {'property': 'nodetype', 'type': 'STRING'},
   {'property': 'effectiveTime', 'type': 'STRING'},
   {'property': 'history', 'type': 'STR

In [3]:
print(graph.schema)


        Node properties are the following:
        [{'labels': 'ObjectConcept', 'properties': [{'property': 'nodetype', 'type': 'STRING'}, {'property': 'history', 'type': 'STRING'}, {'property': 'moduleId', 'type': 'STRING'}, {'property': 'definitionStatusId', 'type': 'STRING'}, {'property': 'effectiveTime', 'type': 'STRING'}, {'property': 'id', 'type': 'STRING'}, {'property': 'sctid', 'type': 'STRING'}, {'property': 'FSN', 'type': 'STRING'}, {'property': 'active', 'type': 'STRING'}]}, {'labels': 'Description', 'properties': [{'property': 'typeId', 'type': 'STRING'}, {'property': 'term', 'type': 'STRING'}, {'property': 'id128bit', 'type': 'STRING'}, {'property': 'descriptionType', 'type': 'STRING'}, {'property': 'refsetId', 'type': 'STRING'}, {'property': 'acceptabilityId', 'type': 'STRING'}, {'property': 'caseSignificanceId', 'type': 'STRING'}, {'property': 'moduleId', 'type': 'STRING'}, {'property': 'nodetype', 'type': 'STRING'}, {'property': 'effectiveTime', 'type': 'STRING'}, {'pr

In [8]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"), graph=graph, verbose=True
)

graph_result = chain.run("What is Atelectasis?")
graph_result



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:ObjectConcept {FSN: 'Atelectasis'})-[:HAS_DESCRIPTION]->(d:Description)
RETURN d.term
Full Context:
[]

> Finished chain.


'Atelectasis is a medical condition that refers to the partial or complete collapse of a lung or a section of a lung. It occurs when the tiny air sacs in the lung, called alveoli, deflate and prevent the lung from fully expanding. This can lead to symptoms such as shortness of breath, chest pain, and coughing. Atelectasis can be caused by various factors, including blockage of the airways, lung diseases, or surgery. Treatment typically involves addressing the underlying cause and may include techniques to re-expand the affected lung tissue.'

In [9]:
chain.run("What are the symptoms of Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:ObjectConcept)-[:HAS_ROLE_GROUP]->(rg:RoleGroup)-[:INTERPRETS]->(s:ObjectConcept)
WHERE a.FSN = 'Atelectasis'
MATCH (s)-[:HAS_ROLE_GROUP]->(rg2:RoleGroup)-[:ASSOCIATED_MORPHOLOGY]->(m:ObjectConcept)
RETURN m.FSN AS Symptom
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question. It would be best to consult a medical professional for accurate information on the symptoms of Atelectasis."

In [10]:
chain.run("What are the causes of Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Description)-[:HAS_ROLE_GROUP]->(rg:RoleGroup)-[:DUE_TO]->(c:ObjectConcept)
WHERE d.term = 'Atelectasis'
RETURN c
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question about the causes of Atelectasis."

In [37]:
documents = text_splitter.create_documents([raw_documents[0].page_content])

In [25]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI

In [38]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [39]:
graph = index_creator.from_text(documents[0].page_content)

In [28]:
graph

In [43]:
graph.draw_graphviz()

[('Atelectasis', 'a lung', 'is the collapse or closure of'),
 ('Atelectasis', 'reduced or absent gas exchange', 'results in'),
 ('Atelectasis', 'unilateral', 'is usually'),
 ('Atelectasis', 'part or all of one lung', 'affects'),
 ('Atelectasis', 'the alveoli are deflated down', 'is a condition where')]

In [13]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI

In [33]:
from langchain. graphs.neo4j_graph import Neo4jGraph

In [44]:
from IPython.display import SVG
graph.draw_graphviz(layout="dot", filename="web.svg")
SVG('web.svg')

ImportError: Could not import pygraphviz python package. Please install it with:`pip install pygraphviz`.

In [14]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [15]:
neo4j_vector = Neo4jVector.from_documents(
    documents,
    # hf,
    OpenAIEmbeddings(),
    url=NOE4J_URL,
    username="neo4j",
    password=NOE4J_PASSWORD,
)

In [16]:
query = "What is atelectasis?"
vector_results = neo4j_vector.similarity_search(query, k=2)
for i, res in enumerate(vector_results):
    print(res.page_content)
    if i != len(vector_results)-1:
        print()
vector_result = vector_results[0].page_content

Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange.  It is usually unilateral, affecting part or all of one lung. It is a condition where the alveoli are deflated down

Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange.  It is usually unilateral, affecting part or all of one lung. It is a condition where the alveoli are deflated down


In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

In [18]:
graph = Neo4jGraph(
        url = NOE4J_URL,
    username = 'neo4j',
    password = NOE4J_PASSWORD
)

print(graph.schema)


        Node properties are the following:
        [{'labels': 'Chunk', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'text', 'type': 'STRING'}, {'property': 'embedding', 'type': 'LIST'}]}]
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [19]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

graph_result = chain.run("What are the symptoms of Atelectasis")
graph_result



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Chunk)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE c.text = 'Atelectasis'
RETURN s.property
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question. Atelectasis is a medical condition characterized by the collapse or closure of a lung or a part of it, resulting in reduced or absent gas exchange. The symptoms of atelectasis can vary depending on the severity and location of the collapse. Common symptoms may include shortness of breath, rapid breathing, chest pain, coughing, and low oxygen levels. However, it is always best to consult a healthcare professional for an accurate diagnosis and appropriate treatment."